In [1]:
import pandas as pd
import re
import numpy as np
train_on_gpu=False

In [4]:
data = pd.read_csv("./data/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data["review"] = data["review"].apply(lambda x:x.lower())

In [7]:
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [8]:
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [9]:
data["review"] = data["review"].apply(lambda x:re.sub("["+punctuation+"]","",x))

In [10]:
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production br br the filmin...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive


In [11]:
from collections import Counter

In [12]:
reviews = data["review"].values
words = " ".join(reviews).split()

In [13]:
count_words = Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)

In [14]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
int_to_vocab = {i:w for (w,i) in vocab_to_int.items()}

In [15]:
reviews_int = []

for review in reviews:
    r = [vocab_to_int[w] for w in review.split()]
    reviews_int.append(r)


In [16]:
def code(label):
    if label=="positive":
        return 1
    if label=="negative":
        return 0

In [17]:
labels = data["sentiment"].apply(lambda x:code(x)).values

In [18]:
def pad_features(reviews_int, seq_length):
    features = np.zeros((len(reviews_int),seq_length),dtype=int)
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        if review_len < seq_length:
            zeros = list(np.zeros(seq_length-review_len))
            new = zeros+review
        elif review_len>seq_length:
            new = review[0:seq_length]
            
        features[i,:] = np.array(new)
        
    return features

In [19]:
padded_reviews = pad_features(reviews_int,50)

In [20]:
split_frac = 0.8
len_feat = padded_reviews.shape[0]
train_x = padded_reviews[0:int(split_frac*len_feat)]
train_y = labels[0:int(split_frac*len_feat)]
remaining_x = padded_reviews[int(split_frac*len_feat):]
remaining_y = labels[int(split_frac*len_feat):]
valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
valid_y = remaining_y[0:int(len(remaining_y)*0.5)]
test_x = remaining_x[int(len(remaining_x)*0.5):]
test_y = remaining_y[int(len(remaining_y)*0.5):]

In [21]:
import torch
from torch.utils.data import DataLoader, TensorDataset
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
# dataloaders
batch_size = 50
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [22]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 50])
Sample input: 
 tensor([[   29,     3,    62,  ...,     1,  1080,   439],
        [24699, 10852,     6,  ..., 20208, 34318, 58341],
        [    8,     6,   203,  ...,  5032,  2624,     4],
        ...,
        [    1,    19,     6,  ...,   310, 99875,  8683],
        [   21,   243,    43,  ...,    64,   177,    11],
        [  255,   105,    10,  ..., 21622,     7,   835]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
        1, 1])


In [26]:
import torch.nn as nn

class SentimentLSTM(nn.Module):
    
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        
        super().__init__()
        
        self.output_size = output_size
        self.n_layers= n_layers
        self.hidden_dim = hidden_dim
        
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            dropout=drop_prob,
                            batch_first=True # inforce the input and output to be of shape (batch_size)
                           )
        
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        
    
    def forward ( self, x):
        
        batch_size = x.size(0)
        
        embeds = self.embedding(x)
        lstm_out, (hidden, c) = self.lstm(embeds)
        #lstm_out = lstm_out[:,-1,:].view(batch_size,self.hidden_dim)
        
        out = self.dropout(hidden)
        out = self.fc(out)
        sig_out = self.sig(out)
        
        return sig_out
    
    

In [27]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 25
hidden_dim = 128
n_layers = 1
net = SentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0)
print(net)

SentimentLSTM(
  (embedding): Embedding(181698, 25)
  (lstm): LSTM(25, 128, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [28]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


# training params

epochs = 5 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        inputs = inputs.type(torch.LongTensor)
        output = net(inputs)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                inputs = inputs.type(torch.LongTensor)
                output = net(inputs)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/5... Step: 100... Loss: 0.695421... Val Loss: 0.691545
Epoch: 1/5... Step: 200... Loss: 0.691282... Val Loss: 0.689838
Epoch: 1/5... Step: 300... Loss: 0.658356... Val Loss: 0.671464
Epoch: 1/5... Step: 400... Loss: 0.659224... Val Loss: 0.672813
Epoch: 1/5... Step: 500... Loss: 0.659676... Val Loss: 0.665162
Epoch: 1/5... Step: 600... Loss: 0.646048... Val Loss: 0.660176
Epoch: 1/5... Step: 700... Loss: 0.747872... Val Loss: 0.689083
Epoch: 1/5... Step: 800... Loss: 0.675645... Val Loss: 0.673713
Epoch: 2/5... Step: 900... Loss: 0.672571... Val Loss: 0.657528
Epoch: 2/5... Step: 1000... Loss: 0.620488... Val Loss: 0.675691
Epoch: 2/5... Step: 1100... Loss: 0.590378... Val Loss: 0.652540
Epoch: 2/5... Step: 1200... Loss: 0.648892... Val Loss: 0.644490
Epoch: 2/5... Step: 1300... Loss: 0.674746... Val Loss: 0.628491
Epoch: 2/5... Step: 1400... Loss: 0.566927... Val Loss: 0.618256
Epoch: 2/5... Step: 1500... Loss: 0.535022... Val Loss: 0.605032
Epoch: 2/5... Step: 1600... Loss: 

In [29]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    inputs = inputs.type(torch.LongTensor)
    output = net(inputs)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.484
Test accuracy: 0.781


In [30]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = re.sub("["+punctuation+"]","",test_review)#''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review("this is bad")
print(test_ints)


# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)


# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())


def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output = net(feature_tensor)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

[[10, 6, 82]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0 10  6 82]]
torch.Size([1, 200])


In [31]:
test_review = 'This movie had the worst acting and the dialogue was so terrible. I hated it.'
seq_length=200 # good to use the length that was trained on
predict(net, test_review, seq_length)

Prediction value, pre-rounding: 0.010006
Negative review detected.


In [32]:
words = "hello hi goodbye eat food bad good fantastic horrible"
words_list = words.split()
ints = tokenize_review(words)
vectors = net.embedding(torch.tensor(ints))

In [33]:
features, labels = vectors[0],words_list
dots = torch.zeros(len(labels),1,1,1)

In [34]:
labels

['hello',
 'hi',
 'goodbye',
 'eat',
 'food',
 'bad',
 'good',
 'fantastic',
 'horrible']

In [35]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('log_dir/experiment1')

In [36]:

writer.add_embedding(features,
                    metadata=labels,
                     label_img=dots
                    )
writer.close()


In [38]:
test_review = 'The movie is so boring and not nice i did not understand it this is good for me anyway'
seq_length=200 # good to use the length that was trained on
predict(net, test_review, seq_length)

Prediction value, pre-rounding: 0.115576
Negative review detected.
